# CNN 教學筆記本
在這本筆記本中，將利用貓、狗圖片資料集訓練一個簡易的影像分類器。

## 程式語言及相關的Library
* Python 
* Numpy
* Pytorch

# 1. 導入相關的Library

In [1]:
import os
import torch
import torchvision
from torch.autograd import Variable
import numpy as np
from data.DataLoader import *

# 2. 導入 貓、狗圖片資料集
簡介：Kaggle在2013年舉辦了辨識圖片中是貓或狗的比賽，對於人來說要辨識貓或狗不是一件難事，但對於電腦而言可不是一件輕鬆的任務。在該場競賽中，參賽者可以開發任意演算法，我們擷取比賽資料集中的一部分資料作為訓練ＣＮＮ的資料。
![woof_meow](./utils/notebook_pic/woof_meow.jpg)
本資料集一共包含2種類別的圖片，包含
* 1 : Cat
* 2 : Dog

In [2]:
from torch.utils import data
train_data = CatDog_Loader(data_root = 'data',train = True)
train_dataloader = data.DataLoader(dataset=train_data,batch_size = 10, shuffle=True)

test_data = CatDog_Loader(data_root = 'data',train = False)
test_dataloader = data.DataLoader(dataset=test_data,batch_size = 4, shuffle=True)

# 3. 定義CNN模型 : LeNet
LeNet是公認的CNN始祖，由Yann LeCun於1998年提出，同時也是目前CNN的hello world。
## LeNet架構、特性可以歸納為 :  
* 卷積層(convolution layer) : 掃描圖像上的特徵、紋理及色彩資料。，並將與
* 池化層(pooling layer) : 減輕從卷積層掃描之特徵。
* 全連接層(fully connective layer) : 藉由萃取後的圖像特徵，進行分類。
* 激活層(activation layer) : 將結果之正相關特徵激活。

![LeNet](./utils/notebook_pic/lenet.png)

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self,num_classes = 2):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, num_classes)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
    
Net = LeNet(num_classes = 2)
Net.train()
Net.cuda()
print("LeNet結構：")
print(Net)

LeNet結構：
LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


# 4. 設定訓練細節
* 訓練次數 : 預設為10
* 設定Loss function : 預設為Cross-Entropy Loss
* 設定Optimizer : 預設為SGD
 

In [4]:
import torch.optim as optim
EPOCH = 20
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(Net.parameters(), lr=0.001, momentum=0.9)

# 5. 進行訓練

In [5]:
for epoch in range(EPOCH): 
    print('Epoch [', epoch,' / ', EPOCH ,'] : ')
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # 取得訓練資料
        inputs, labels = data
        inputs = Variable(inputs).cuda()
        labels = Variable(labels).cuda()
        # 初始化optimizer
        optimizer.zero_grad()
        # 資料送至AlexNet : Forward
        outputs = Net(inputs)
        # 計算Loss
        loss = criterion(outputs, labels)
        # 修正網路並調整Optimizer
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 20 == 19:    #每40步輸出訓練成果
            print('[%d, %5d] loss: %.3f' %
                  (i-18, i + 1, running_loss / 19))
            running_loss = 0.0

print('結束訓練')

Epoch [ 0  /  20 ] : 
[1,    20] loss: 0.802
Epoch [ 1  /  20 ] : 
[1,    20] loss: 0.728
Epoch [ 2  /  20 ] : 
[1,    20] loss: 0.707
Epoch [ 3  /  20 ] : 
[1,    20] loss: 0.670
Epoch [ 4  /  20 ] : 
[1,    20] loss: 0.641
Epoch [ 5  /  20 ] : 
[1,    20] loss: 0.687
Epoch [ 6  /  20 ] : 
[1,    20] loss: 0.575
Epoch [ 7  /  20 ] : 
[1,    20] loss: 0.534
Epoch [ 8  /  20 ] : 
[1,    20] loss: 0.533
Epoch [ 9  /  20 ] : 
[1,    20] loss: 0.532
Epoch [ 10  /  20 ] : 
[1,    20] loss: 0.447
Epoch [ 11  /  20 ] : 
[1,    20] loss: 0.469
Epoch [ 12  /  20 ] : 
[1,    20] loss: 0.363
Epoch [ 13  /  20 ] : 
[1,    20] loss: 0.315
Epoch [ 14  /  20 ] : 
[1,    20] loss: 0.193
Epoch [ 15  /  20 ] : 
[1,    20] loss: 0.142
Epoch [ 16  /  20 ] : 
[1,    20] loss: 0.049
Epoch [ 17  /  20 ] : 
[1,    20] loss: 0.152
Epoch [ 18  /  20 ] : 
[1,    20] loss: 0.063
Epoch [ 19  /  20 ] : 
[1,    20] loss: 0.159
結束訓練


# 訓練20 EPOCH 後的細節
.<br />
.<br />
.<br />
Epoch [ 17  /  20 ] : <br />
[1,    20] loss: 0.152<br />
Epoch [ 18  /  20 ] : <br />
[1,    20] loss: 0.063<br />
Epoch [ 19  /  20 ] : <br />
[1,    20] loss: 0.159<br />

# 6. 測試訓練後的模型

In [7]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
classes = ('cat','dog')
with torch.no_grad():
    
    for data in test_dataloader:
        images, labels = data
        images = Variable(images).cuda()
        
        outputs = Net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted.data.cpu() == labels).squeeze()
        
        for i in range(2):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of   cat : 100 %
Accuracy of   dog : 100 %


# 訓練20 EPOCH 後的成果
Accuracy of   cat : 100 % <br />
Accuracy of   dog : 100 % <br />